In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

# Definice vlastního datasetu pro čtení X, Y z názvu souboru
class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.1, 0.1, 0.1, 0.1)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = PIL.Image.open(image_path)
        
        # Parsování názvu souboru: xy_<x>_<y>_<uuid>.jpg
        filename = os.path.basename(image_path)
        try:
            # Rozdělíme podle podtržítka a vezmeme 2. a 3. hodnotu
            parts = filename.split('_')
            x = float(parts[1])
            y = float(parts[2])
        except:
            # Fallback kdyby byl název špatně
            x = 0.0
            y = 0.0
        
        # Data Augmentation (náhodné překlopení obrazu)
        # Pokud se robot učí zatáčet, musíme při překlopení obrazu otočit i souřadnici X!
        if self.random_hflips and float(np.random.random()) > 0.5:
            image = image.transpose(PIL.Image.FLIP_LEFT_RIGHT)
            x = -x  # Invertujeme směr zatáčení
            
        x = torch.tensor([x, y]).float()
        
        # Transformace obrazu (stejné jako u klasifikace)
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, x

In [3]:
# Načtení datasetu
dataset = XYDataset('dataset_xy', random_hflips=False)

# Rozdělení na trénovací a testovací sadu (90% trénink, 10% test)
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

# Vytvoření DataLoaderů
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=4
)

print(f"Počet trénovacích snímků: {len(train_dataset)}")
print(f"Počet testovacích snímků: {len(test_dataset)}")

Počet trénovacích snímků: 315
Počet testovacích snímků: 35


In [4]:
# Použijeme ResNet18
model = models.resnet18(pretrained=True)

# ZMĚNA: Výstupní vrstva má 2 výstupy (X, Y) místo kategorií
model.fc = torch.nn.Linear(512, 2)

# Přesun na GPU
device = torch.device('cuda')
model = model.to(device)

In [5]:
NUM_EPOCHS = 50
BEST_MODEL_PATH = 'best_steering_model_xy.pth'
best_loss = 1e9  # Inicializujeme na vysokou hodnotu

# Optimizer (Adam bývá pro regresi lepší než SGD, ale SGD funguje taky)
optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train() # Přepnutí do trénovacího režimu
    train_loss = 0.0
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        
        # ZMĚNA: Používáme MSELoss (Mean Squared Error)
        loss = F.mse_loss(outputs, labels)
        
        loss.backward()
        optimizer.step()
        train_loss += float(loss)
        
    train_loss /= len(train_loader)
    
    # Validace (Testování)
    model.eval() # Přepnutí do evaluačního režimu
    test_loss = 0.0
    
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += float(loss)
        
    test_loss /= len(test_loader)
    
    print('%d: Train Loss: %f, Test Loss: %f' % (epoch, train_loss, test_loss))
    
    # Ukládáme model, pokud má nejmenší chybu (loss)
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss
        print("  -> Model uložen!")

0: Train Loss: 12015.685413, Test Loss: 9834.287305
  -> Model uložen!
1: Train Loss: 6903.724542, Test Loss: 4586.186670
  -> Model uložen!
2: Train Loss: 3076.781735, Test Loss: 1287.922559
  -> Model uložen!
3: Train Loss: 1376.460628, Test Loss: 796.517764
  -> Model uložen!
4: Train Loss: 945.237761, Test Loss: 632.410321
  -> Model uložen!
5: Train Loss: 745.004252, Test Loss: 440.730573
  -> Model uložen!
6: Train Loss: 560.766707, Test Loss: 381.738396
  -> Model uložen!
7: Train Loss: 634.528408, Test Loss: 465.421857
8: Train Loss: 452.261987, Test Loss: 359.135971
  -> Model uložen!
9: Train Loss: 381.923516, Test Loss: 369.630971
10: Train Loss: 330.686183, Test Loss: 432.524342
11: Train Loss: 308.880493, Test Loss: 391.987531
12: Train Loss: 396.906008, Test Loss: 612.630383
13: Train Loss: 336.760387, Test Loss: 458.709296
14: Train Loss: 222.523568, Test Loss: 392.602539
15: Train Loss: 200.814059, Test Loss: 314.237074
  -> Model uložen!
16: Train Loss: 186.980467, Tes